# Quantum Stock Price Prediction with PennyLane

This notebook demonstrates a simple quantum machine learning model that predicts stock prices using a variational quantum circuit.

**What we'll do:**
1. Load Tesla stock price data
2. Normalize the data for quantum processing
3. Build a quantum circuit
4. Train the model
5. Predict tomorrow's price
6. Visualize the results

---

## 1. Import Libraries

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

---

## 2. Load Stock Price Data

We're using recent Tesla (TSLA) closing prices. You can also fetch live data using `yfinance` if you have it installed.

```python
# OPTION A: Real-world data (Requires 'pip install yfinance')
# import yfinance as yf
# ticker = yf.Ticker("TSLA")
# data = ticker.history(period="1mo")['Close'].values
# prices = np.array(data[-15:])
```

In [ ]:
# Tesla closing prices from December 2025
prices = np.array([
    430.14, 429.24, 446.74, 454.53, 455.00, 
    439.58, 445.17, 451.45, 446.89, 458.96, 
    475.31
])

days = np.arange(len(prices))

print(f"Loaded {len(prices)} days of stock data")
print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}")

---

## 3. Normalize the Data

Quantum circuits work best with data in specific ranges:
- **Input (days)**: Normalized to [0, π]
- **Output (prices)**: Normalized to [-0.9, 0.9]

This scaling is crucial because rotation gates use angles, not raw dollar values.

In [ ]:
# Normalize Days (Input) to range [0, Pi]
x_norm = (days - days.min()) / (days.max() - days.min()) * np.pi

# Normalize Prices (Target) to range [-0.9, 0.9]
p_min, p_max = prices.min(), prices.max()
y_norm = (prices - p_min) / (p_max - p_min) * 1.8 - 0.9

print(f"Normalized input range: {x_norm.min():.4f} to {x_norm.max():.4f}")
print(f"Normalized output range: {y_norm.min():.4f} to {y_norm.max():.4f}")

---

## 4. Build the Quantum Circuit

Our circuit has three parts:
1. **Data Encoding**: `RX` gate encodes the day as a rotation
2. **Trainable Model**: `Rot` gate with 3 learnable parameters
3. **Measurement**: Measures the qubit in the Z-basis (returns value between -1 and 1)

In [ ]:
# Initialize quantum device
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit(params, x):
    # Data Encoding: Map day index to an angle
    qml.RX(x, wires=0)
    
    # Trainable Model: 3 rotation parameters
    qml.Rot(params[0], params[1], params[2], wires=0)
    
    # Measurement: Returns value between -1 and 1
    return qml.expval(qml.PauliZ(0))

# Visualize the circuit
print("Circuit structure:")
print(qml.draw(circuit)(np.array([0.1, 0.2, 0.3]), 1.0))

---

## 5. Define the Cost Function

We'll use Mean Squared Error (MSE) to measure how well our predictions match the actual prices.

In [ ]:
def cost(params, x, y):
    predictions = [circuit(params, xv) for xv in x]
    return np.mean((predictions - y) ** 2)

---

## 6. Train the Model

We'll use the Adam optimizer to train our quantum circuit for 100 steps.

In [ ]:
# Initialize random parameters
np.random.seed(42)  # For reproducibility
params = np.random.uniform(0, np.pi, 3, requires_grad=True)

# Initialize optimizer
opt = qml.AdamOptimizer(stepsize=0.1)

print(f"Training on {len(prices)} days of stock data...")
print("-" * 50)

# Training loop
for i in range(100):
    params, _, _ = opt.step(cost, params, x_norm, y_norm)
    
    if (i+1) % 20 == 0:
        current_cost = cost(params, x_norm, y_norm)
        print(f"Step {i+1:3d}: Cost = {current_cost:.6f}")

print("-" * 50)
print("Training complete!")

---

## 7. Predict Tomorrow's Price

Now let's use our trained model to predict the next day's closing price.

In [ ]:
# Predict for the next day (one step beyond our data)
next_day_norm = np.pi + (np.pi / len(days))

# Get normalized prediction
prediction_norm = circuit(params, next_day_norm)

# Denormalize to get actual dollar price
predicted_price = ((prediction_norm + 0.9) / 1.8) * (p_max - p_min) + p_min

print(f"📈 Predicted Stock Price for Day {len(days)+1}: ${predicted_price:.2f}")
print(f"   Last known price (Day {len(days)}): ${prices[-1]:.2f}")
print(f"   Change: ${predicted_price - prices[-1]:+.2f}")

---

## 8. Visualize Results

Let's plot the actual data, our quantum model's fit, and the prediction.

In [ ]:
# Calculate predictions for all training points
y_pred_norm = [circuit(params, x) for x in x_norm]

# Denormalize predictions back to dollar values
y_pred_real = [((y + 0.9) / 1.8) * (p_max - p_min) + p_min for y in y_pred_norm]

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(days, prices, color='black', s=100, label='Actual Data', zorder=3)
plt.plot(days, y_pred_real, color='blue', linewidth=2, label='Quantum Fit')
plt.scatter([len(days)], [predicted_price], color='red', marker='x', 
            s=200, linewidth=3, label='Prediction', zorder=4)

plt.legend(fontsize=12)
plt.title("Quantum Stock Prediction (Simple Regression)", fontsize=14, fontweight='bold')
plt.xlabel("Day", fontsize=12)
plt.ylabel("Price ($)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## Summary

✅ **What we accomplished:**
- Built a simple quantum regression model using PennyLane
- Trained it on 11 days of Tesla stock prices
- Made a prediction for the next day

⚠️ **Important notes:**
- This is a **toy example** for educational purposes
- Real stock prediction requires far more sophisticated models
- Quantum advantage for such problems is still an open research question

🔬 **Next steps to explore:**
- Try more qubits and deeper circuits
- Add more features (volume, other indicators)
- Compare with classical ML models